INDICE DE ALFABETIZACIÓN<br>
extraido WDB - "SE.ADT.LITR.ZS" - DB=2

In [1]:
#importo librerias
import wbgapi as wb
import pandas as pd

In [3]:
#Filtro por pais
paises = ['ATG','ARG','BRB','BLZ','BOL','BRA','CAN','CHL','COL','CRI','CUB','DMA','DOM','ECU','SLV','GRD','GTM','GUY','HTI','HND','JAM','MEX','NIC','PAN','PRY','PER','KNA','LCA','VCT','SUR','BHS','TTO','USA','URY','VEN']

In [4]:
#INDICE ALFABETIZACION - extraido de WDB - "SE.ADT.LITR.ZS" - DB=2

#CREACION DE DATAFRAME
wb.db= 2
tbl_alfabetizacion_porc = wb.data.DataFrame('SE.ADT.LITR.ZS', paises, time=range(2000, 2021), labels=True, columns="series").reset_index()
tbl_alfabetizacion_porc.drop(["time","Country"], axis=1, inplace=True)
tbl_alfabetizacion_porc.columns=["Pais_ID","Year","Indice_Alfab."]

In [5]:
tbl_alfabetizacion_porc.head()

,Pais_ID,Year,Indice_Alfab.
0,VEN,2020,NaN
1,VEN,2019,NaN
2,VEN,2018,NaN
3,VEN,2017,NaN
4,VEN,2016,97.12709


In [6]:
tbl_alfabetizacion_porc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Pais_ID        735 non-null    object 
 1   Year           735 non-null    object 
 2   Indice_Alfab.  200 non-null    float64
dtypes: float64(1), object(2)
memory usage: 17.4+ KB


In [7]:
#CONTAMOS VALORES NULOS
tbl_alfabetizacion_porc.isnull().sum()

Pais_ID            0
Year               0
Indice_Alfab.    535
dtype: int64

HAY 535 datos faltantes, algunos paises tienen todos sus datos faltantes<br>
Vamos a identificarlos

In [8]:
print("Paises con TODOS sus datos nulos")
paises_nulos=[]
for i in paises:
    mask = tbl_alfabetizacion_porc["Pais_ID"] == i
    df_filtrada = tbl_alfabetizacion_porc[mask]
    a = df_filtrada.iloc[:,2].isnull().sum()
    if a == 21:
        paises_nulos.append(i)
print(paises_nulos)

Paises con TODOS sus datos nulos
['CAN', 'DMA', 'KNA', 'LCA', 'VCT', 'BHS', 'USA']


Imputaremos un valor para cada uno de ellos, para luego predecir el resto de la tabla<br>
Fuente: https://worldpopulationreview.com/country-rankings/literacy-rate-by-country<br>

CAN = Canada, Año 2003, Ind. Alfabetizacion: 99.00% <br>
DMA = Dominica, Año 2003, Ind. Alfabetizacion: 94.00%<br>
KNA = Saint Kitts and Nevis, Año 2003, Ind. Alfabetizacion: 97.80%<br>
LCA = Santa Lucia, Año 2003, Ind. Alfabetizacion: 90.10%<br>
VCT = Saint Vincent and the Grenadines, Año: 2003, Ind. Alfabetizacion: 93.00%<br>
BHS = Bahamas, Año: 2003, Ind. Alfabetizacion: 95.60%<br>
USA = Estados Unidos, Año: 2003, Ind. Alfabetizacion: 99.00%

In [9]:
#creamos un dataframe con los valores a cargar
imp=[99.00, 94.00, 97.80, 90.10, 93.00, 95.60, 99.00]
df_carga=pd.DataFrame(list(zip(paises_nulos, imp)), columns= ["Pais","IND"])
df_carga

,Pais,IND
0,CAN,99.0
1,DMA,94.0
2,KNA,97.8
3,LCA,90.1
4,VCT,93.0
5,BHS,95.6
6,USA,99.0


In [10]:
#Cargamos los valores en cada celda correspondiente al pais y año
for i in paises_nulos:
    mask1 = (tbl_alfabetizacion_porc["Pais_ID"] == i) & (tbl_alfabetizacion_porc["Year"] == "2003")
    mask2 = df_carga["Pais"] == i
    tbl_alfabetizacion_porc.loc[mask1, "Indice_Alfab."] = df_carga["IND"].loc[mask2].values[0]
    print (tbl_alfabetizacion_porc[mask1])

    Pais_ID  Year  Indice_Alfab.
605     CAN  2003           99.0
    Pais_ID  Year  Indice_Alfab.
500     DMA  2003           94.0
    Pais_ID  Year  Indice_Alfab.
185     KNA  2003           97.8
    Pais_ID  Year  Indice_Alfab.
164     LCA  2003           90.1
    Pais_ID  Year  Indice_Alfab.
143     VCT  2003           93.0
    Pais_ID  Year  Indice_Alfab.
101     BHS  2003           95.6
   Pais_ID  Year  Indice_Alfab.
59     USA  2003           99.0


In [11]:
#Corroboramos que no haya mas paises con todos sus datos nulos
print("Paises con TODOS sus datos nulos")
paises_nulos_2=[]
for i in paises:
    mask = tbl_alfabetizacion_porc["Pais_ID"] == i
    df_filtrada = tbl_alfabetizacion_porc[mask]
    a = df_filtrada.iloc[:,2].isnull().sum()
    if a == 21:
        paises_nulos_2.append(i)
print(paises_nulos_2)

Paises con TODOS sus datos nulos
[]


In [12]:
#Corroboramos nuevamente datos nulos, vemos que disminuyo de 535 a 528
tbl_alfabetizacion_porc.isnull().sum()

Pais_ID            0
Year               0
Indice_Alfab.    528
dtype: int64

In [13]:
#Separamos los datos por pais para observar su comportamiento
for i in paises:
    print(tbl_alfabetizacion_porc[tbl_alfabetizacion_porc["Pais_ID"] == i])

    Pais_ID  Year  Indice_Alfab.
714     ATG  2020            NaN
715     ATG  2019            NaN
716     ATG  2018            NaN
717     ATG  2017            NaN
718     ATG  2016            NaN
719     ATG  2015      98.949997
720     ATG  2014            NaN
721     ATG  2013            NaN
722     ATG  2012            NaN
723     ATG  2011      98.949997
724     ATG  2010            NaN
725     ATG  2009            NaN
726     ATG  2008            NaN
727     ATG  2007            NaN
728     ATG  2006            NaN
729     ATG  2005            NaN
730     ATG  2004            NaN
731     ATG  2003            NaN
732     ATG  2002            NaN
733     ATG  2001      98.949997
734     ATG  2000            NaN
    Pais_ID  Year  Indice_Alfab.
693     ARG  2020            NaN
694     ARG  2019            NaN
695     ARG  2018      99.003868
696     ARG  2017            NaN
697     ARG  2016      99.125008
698     ARG  2015      99.179962
699     ARG  2014      98.993889
700     AR

In [14]:
#Se aplicará interpolacion para llenar los datos faltantes por país:
for i in paises:
    temp = tbl_alfabetizacion_porc[tbl_alfabetizacion_porc['Pais_ID'] == i]
    temp = temp.interpolate()
    tbl_alfabetizacion_porc[tbl_alfabetizacion_porc['Pais_ID'] == i] = temp

In [15]:
#Separamos los datos por pais para observar el resultado
for i in paises:
    print(tbl_alfabetizacion_porc[tbl_alfabetizacion_porc["Pais_ID"] == i])

    Pais_ID  Year  Indice_Alfab.
714     ATG  2020            NaN
715     ATG  2019            NaN
716     ATG  2018            NaN
717     ATG  2017            NaN
718     ATG  2016            NaN
719     ATG  2015      98.949997
720     ATG  2014      98.949997
721     ATG  2013      98.949997
722     ATG  2012      98.949997
723     ATG  2011      98.949997
724     ATG  2010      98.949997
725     ATG  2009      98.949997
726     ATG  2008      98.949997
727     ATG  2007      98.949997
728     ATG  2006      98.949997
729     ATG  2005      98.949997
730     ATG  2004      98.949997
731     ATG  2003      98.949997
732     ATG  2002      98.949997
733     ATG  2001      98.949997
734     ATG  2000      98.949997
    Pais_ID  Year  Indice_Alfab.
693     ARG  2020            NaN
694     ARG  2019            NaN
695     ARG  2018      99.003868
696     ARG  2017      99.064438
697     ARG  2016      99.125008
698     ARG  2015      99.179962
699     ARG  2014      98.993889
700     AR

In [16]:
#Sola hay datos faltantes para los ultimos años, aplicaremos nuevamente interpolacion pero con direccion de atras hacia adelante
# para que llene los valores faltantes
for i in paises:
    temp = tbl_alfabetizacion_porc[tbl_alfabetizacion_porc['Pais_ID'] == i]
    temp = temp.interpolate(method="linear",limit_direction="backward")
    tbl_alfabetizacion_porc[tbl_alfabetizacion_porc['Pais_ID'] == i] = temp

In [17]:
#No hay valores null
tbl_alfabetizacion_porc.isnull().sum()

Pais_ID          0
Year             0
Indice_Alfab.    0
dtype: int64

In [18]:
#vemos el resultado final
for i in paises:
    print(tbl_alfabetizacion_porc[tbl_alfabetizacion_porc["Pais_ID"] == i])

    Pais_ID  Year  Indice_Alfab.
714     ATG  2020      98.949997
715     ATG  2019      98.949997
716     ATG  2018      98.949997
717     ATG  2017      98.949997
718     ATG  2016      98.949997
719     ATG  2015      98.949997
720     ATG  2014      98.949997
721     ATG  2013      98.949997
722     ATG  2012      98.949997
723     ATG  2011      98.949997
724     ATG  2010      98.949997
725     ATG  2009      98.949997
726     ATG  2008      98.949997
727     ATG  2007      98.949997
728     ATG  2006      98.949997
729     ATG  2005      98.949997
730     ATG  2004      98.949997
731     ATG  2003      98.949997
732     ATG  2002      98.949997
733     ATG  2001      98.949997
734     ATG  2000      98.949997
    Pais_ID  Year  Indice_Alfab.
693     ARG  2020      99.003868
694     ARG  2019      99.003868
695     ARG  2018      99.003868
696     ARG  2017      99.064438
697     ARG  2016      99.125008
698     ARG  2015      99.179962
699     ARG  2014      98.993889
700     AR